In [1]:
# Numerical/scientific computing packages.
import numpy as np
import scipy
import copy
import pandas as pd
import seaborn as sns
import csv

# Machine learning package.
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error, mean_absolute_error


# Useful for saving our models.
import pickle

# Plotting packages.
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Sets the matplotlib backend for the notebook.
# sets the backend of matplotlib to the 'inline' backend: 
# With this backend, the output of plotting commands is 
# displayed inline within the Jupyter notebook, 
# directly below the code cell that produced it
%matplotlib inline 

print('Finished successfully loading packages')

Finished successfully loading packages


### QUESTION 1: Using multiple linear regression: What is the best predictor of total annual compensation, how much variance is explained by this predictor vs. the full multiple regression model?

In [2]:
def simple_linear_regression(data: any, title: str = "TITLE NOT SET", x_axis_label: str = "X AXIS NOT SET", y_axis_label: str = "Y AXIS NOT SET"):
    pass

def multiple_linear_regression(data: any, title: str = "TITLE NOT SET", x_axis_label: str = "X AXIS NOT SET", y_axis_label: str = "Y AXIS NOT SET"):
    pass

### QUESTION 2: Using ridge regression to do the same as in 1): How does the model change or improve compared to OLS? What is the optimal lambda?

In [3]:
def foo():
    pass

### QUESTION 3: Using Lasso regression to do the same as in 1): How does the model change now? How many of the predictor betas are shrunk to exactly 0? What is the optimal lambda now?

In [4]:
def foo():
    pass

### QUESTION 4: There is controversy as to the existence of a male/female gender pay gap in tech job compensation. Build a logistic regression model (with gender as the outcome variable) to see if there is an appreciable beta associated with total annual compensation with and without controlling for other factors.

In [5]:
def foo():
    pass

### QUESTION 5: Build a logistic regression model to see if you can predict high and low pay from years of relevant experience, age, height, SAT score and GPA, respectively.

In [6]:
def foo():
    pass

### EXTRA CREDIT (A): Is salary, height or age normally distributed? Does this surprise you? Why or why not?

In [7]:
def foo():
    pass

### EXTRA CREDIT (B): Tell us something interesting about this dataset that is not already covered by the questions above and that is not obvious.

In [8]:
def foo():
    pass